nbviewer.jupyter.org/format/slides/gist/leechanwoo/

In [ ]:
from IPython.display import Image

# Deep Learning Day 2.

## Scenario 1. FFNN (Binary): Feed Forward Neural Network
 * Dataset
 * Reading Dataset File
 * Building Model
 * Model Save/Restore
 * Tensorboard

### Dataset

|label|data0|data1|data2|data3|data4|data5|data6|data7|data8|data9|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|0|0|1|2|3|4|5|6|7|8|9|
|1|9|8|7|6|5|4|3|2|1|0|
|0|0|1|2|3|4|5|6|7|8|9|
|1|9|8|7|6|5|4|3|2|1|0|
|0|0|1|2|3|4|5|6|7|8|9|
|1|9|8|7|6|5|4|3|2|1|0|

### Reading Dataset File

In [ ]:
Image('./day2/pipeline.gif.png')

### Building Model

 * Feed Forward Neural Network(FFNN)
 * Neural Network 모델 중 가장 기본적인 모델


$$ y_0 = w_{00}x_0 + w_{01}x_1 + w_{02}x_2 + w_{03}x_3 ... w_{0n}x_n $$
$$ y_1 = w_{10}x_0 + w_{11}x_1 + w_{12}x_2 + w_{13}x_3 ... w_{1n}x_n $$
$$ y_2 = w_{20}w_0 + w_{21}x_1 + w_{22}x_2 + w_{23}x_3 ... w_{2n}x_n $$
$$ y_3 = w_{30}w_0 + w_{31}x_1 + w_{32}x_2 + w_{33}x_3 ... w_{3n}x_n $$
$$...$$
$$ y_m = w_{m0}x_0 + w_{m1}x_1 + w_{m2}x_2 + w_{m3}x_3 ... w_{mn}x_n $$


$$ \begin{bmatrix} y_0\\y_1\\y_2\\y_3\\...\\y_m \end{bmatrix} = \begin{bmatrix} w_{00}&w_{01}&w_{02}&w_{03}&...&w_{0n} \\ w_{10}&w_{11}&w_{12}&w_{13}&...&w_{1n} \\ w_{20}&w_{21}&w_{22}&w_{23}&...&w_{2n} \\ w_{30}&w_{31}&w_{32}&w_{33}&...&w_{3n} \\ ...&...&...&...&...&... \\ w_{m0}&w_{m1}&w_{m2}&w_{m3}&...&w_{mn} \end{bmatrix} \begin{bmatrix} x_0\\x_1\\x_2\\x_3\\...\\x_n \end{bmatrix} $$

$$ Y^{(1)} = W_1^TX \cdots 1\ layer$$ 

$$ Y^{(2)} = W_2^TY^{(1)} \cdots 2\ layer $$

$$ Y^{(3)} = W_3^TY^{(2)} \cdots 3\ layer $$

 * Neural Net 내부의 Layer는 hidden layer라고 부른다.
 * 각 hidden layer에는 activation function을 취해준다.
 * 여기서는 sigmoid function을 activation function으로 사용
 
$$ \sigma(x) = \frac{1}{1 + e^{-(wx + b)}} $$ <br>

$$ h = \sigma(W^TX) $$

$$ h^{(1)} = \sigma(W_1^TX) \cdots 1\ layer$$ <br>
$$ h^{(2)} = \sigma(W_2^Th^{(1)}) \cdots 2\ layer $$ <br>
$$ h^{(3)} = \sigma(W_3^Th^{(2)}) \cdots 3\ layer $$ <br>
$$ Y = \sigma(W_o^Th^{(3)}) \cdots output\ layer $$ <br>

In [ ]:
Image('./day2/nerual_network_bin.png')

### Model save/restore

In [ ]:
Image('./day2/saver.gif.png')

### Implementation

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import csv

/Users/beomyongnho/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### (1) csv file format으로 데이터 생성

In [2]:
samples = 1000
test_samples = 100
# 이 ipynb 파일이 있는 폴더에 하위폴더로 ffnn_dataset 폴더를 생성해놔야 함. (아니면 디렉토리 주소를 변경해야...)
train_dataset = './ffnn_dataset/train_dataset.csv'
test_dataset = './ffnn_dataset/test_dataset.csv'

# 아래 함수의 파라미터는 train sample 갯수, test sample 갯수와 각각의 sample data를 저장할 디렉토리를 의미함
def write_dataset(samples, test_samples, train_dir, test_dir):
    up = [i for i in range(10)]
    down = [9-i for i in range(10)]

    data = []
    label = []
    for i in range(samples):
        data.append(up)
        data.append(down)
        label.append([1])
        label.append([0])

    with open(train_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(samples-test_samples):
            writer.writerow(label[i] + data[i])
        print('train data is written')

    with open(test_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(test_samples):
            writer.writerow(label[i] + data[i])
        print('test data is written')
        
#write_dataset(1000, 100, train_dataset, test_dataset)

### (2) 데이터 불러오기

일단 첫 번째로 Dataset 객체를 만들어줘야 하는데, 파일 이름을 쓰는 부분이 있다. 
여기에는 지금은 그냥 만들어 둔 csv 파일 경로 하나만 넣어놨지만, 
다수의 파일이 존재 할 경우 ["file_1.csv", "file_2.csv", "file_3.csv" ... "file_n.csv"] 
이렇게 리스트 형태로 넣어줘도 상관없다. 
데이터 셋 메서드를 이용하여 shuffle과 batching을 아래와 같이 해주면 쉽게 데이터를 얻어올 수 있다.

In [3]:
tf.reset_default_graph() # tensorflow의 graph를 리셋하기 위한 라인

# dataset객체를 생성. 파일 경로와 이름을 넣고 batch method로 batching까지 완료(1batch당 data 10개씩)
# 다수의 파일이 존재 할 경우 ["file_1.csv", "file_2.csv", "file_3.csv" ... "file_n.csv"] 이렇게 리스트 형태로 넣어주는 것도 가능
# method를 추가해서 shuffle까지도 구현 가능 
# 예: tf.contrib.data.TextLineDataset(test_dataset).shuffle(100).batch(10)

trainset = tf.contrib.data.TextLineDataset(train_dataset).batch(10)
trainset = trainset.repeat(100)

testset = tf.contrib.data.TextLineDataset(test_dataset).batch(10)
testset = testset.repeat(99999)



# iterator를 만들어주는 부분. 
# dataset 클래스는 내부에서 queue runners가 동작하는데 iterator는 그 queue에다가 dequeue 요청을 주는 역할을 한다고 보면 된다. 
# batch가 실행 될 때마다 다음 batch를 얻어온다.
train_itr = trainset.make_one_shot_iterator()
test_itr = testset.make_one_shot_iterator()
# 여기까지가 iterator 

train_batch = train_itr.get_next()
test_batch = test_itr.get_next()
# 여기까지가 iterator를 받아 다음 batch를 얻어오기 

# text reader로 data를 읽어왔으므로 받아온 data는 string type!
# csv를 decode하기만 하면 우리가 원하는 데이터셋을 얻을 수 있음
# stack과 expand_dims의 활용은 tensorflow docs를 참고
# 아래 코드에서 [[0]]*11는 record_default를 의미하며 missing data 발생 시 이를 채워주는 default value를 설정하는 것
train_batch = tf.decode_csv(train_batch, [[0]]*11) 
test_batch = tf.decode_csv(test_batch, [[0]]*11)

y_ = tf.reshape(train_batch[0], [-1, 1]) 
# 위 라인에서 train_batch[0]은 'column index=0'을 numpy ndarray로 받음. [1 0 ... 0 1] 형태임. 
# [-1, 1]은 reshape 과정에서 행의 갯수는 따로 지정하지 않고 열의 갯수는 1개로 지정함을 의미
test_label = tf.reshape(test_batch[0], [-1, 1])

x = tf.stack(train_batch[1:], axis=1)
test_data = tf.stack(test_batch[1:], axis=1)

# 마지막으로 머신러닝 모델에서 연산을 수행 하기 위해선 이렇게 float type으로 casting 해주면 된다.
# 이 과정을 빼먹으면 dtype = int32가 되어 사칙연산이 제대로 수행되지 않는다.(불가여부는?)
test_label = tf.cast(test_label, tf.float32)
test_input = tf.cast(test_data, tf.float32)
y_ = tf.cast(y_, tf.float32)
x = tf.cast(x, tf.float32)

print(y_)
print(x)
print(test_label)
print(test_input)

Instructions for updating:
Use `tf.data.TextLineDataset`.
Tensor("Cast_2:0", shape=(?, 1), dtype=float32)
Tensor("Cast_3:0", shape=(?, 10), dtype=float32)
Tensor("Cast:0", shape=(?, 1), dtype=float32)
Tensor("Cast_1:0", shape=(?, 10), dtype=float32)


In [ ]:
# define과정에서 데이터는 다음과 같이 출력해서 확인해보는 것이 좋다. 
# with tf.Session() as sess:
#     _a= sess.run(y_)
#     print(_a)
#     print(type(_a))

In [ ]:
def bin_model(x, activation=None, reuse=False):
    # dense method는 fully connected 모델을 생성함을 의미한다. 
    # x는 input data
    # activation은 말 그대로 activation function
    # reuse는 Boolean type keyword parameter로 previous layer의 가중치 재사용 여부를 결정. test data로 검증할 때 reuse 
    layer1 = tf.layers.dense(x, 10, activation=activation, reuse=reuse, name='layer1')
    layer2 = tf.layers.dense(layer1, 10, activation=activation, reuse=reuse, name='layer2')
    layer3 = tf.layers.dense(layer2, 10, activation=activation, reuse=reuse, name='layer3')
    layer4 = tf.layers.dense(layer3, 10, activation=activation, reuse=reuse, name='layer4')
    out = tf.layers.dense(layer4, 1, reuse=reuse, name='out')
    print('layer1 shape {}'.format(layer1.shape))
    print('layer2 shape {}'.format(layer2.shape))
    print('layer3 shape {}'.format(layer3.shape))
    print('layer4 shape {}'.format(layer4.shape))
    print('out shape {}'.format(out.shape))
    return out
    
train_out = bin_model(x, activation=tf.nn.sigmoid)
test_out = bin_model(test_input, activation=tf.nn.sigmoid, reuse=True)
# test data의 경우 training data로 fitting해서 구한 가중치를 그대로 사용해서 결과를 도출해야 하므로 reuse=True가 된다. 

# variable들(즉, weights(가중치)들의 shape을 살펴보기 위한 코드(input과 output의 shape에 따라 달라짐) 
for var in tf.trainable_variables():
    print(var)
    
saver = tf.train.Saver()

In [ ]:
# loss function과 optimizer 정의
loss = tf.losses.sigmoid_cross_entropy(y_, train_out) #loss function과 activation function은 다르다
train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)

In [ ]:
pred = tf.nn.sigmoid(test_out)
accuracy = tf.metrics.accuracy(test_label, tf.round(pred))

In [ ]:
tf.summary.scalar('loss', loss)
merged = tf.summary.merge_all()

In [ ]:
##training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    writer = tf.summary.FileWriter('./ffnn_logs/', sess.graph)
    for i in range(1000):
        try:
            _, _loss, _acc, _summ = sess.run([train_op, loss, accuracy, merged])
            writer.add_summary(_summ, i)
            if i%100 == 0:
                _pred = sess.run(pred)
                print('step: {}, loss: {}, acc: {}'.format(i, _loss, _acc[0]))
        
        except tf.errors.OutOfRangeError:
            break

    saver.save(sess, './ffnn_logs/ffnn')
    
#prediction
with tf.Session() as sess:
    saver.restore(sess, './ffnn_logs/ffnn')
    _pred = sess.run(pred)
    print(_pred)

## Scenario 2. FFNN(Multi-class)
 * Building Model

### Building Model

In [ ]:
Image('./day2/neural_network.png')

### Implementation

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import csv

/Users/beomyongnho/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
samples = 1000
test_samples = 100
train_dataset = './ffnn_multiclass_dataset/train_dataset.csv'
test_dataset = './ffnn_multiclass_dataset/test_dataset.csv'

up = [i for i in range(10)]
down = [9-i for i in range(10)]
flat = [5 for i in range(10)]

data = []
label = []
for i in range(samples):
    data.append(up)
    data.append(flat)
    data.append(down)
    label.append([0])
    label.append([1])
    label.append([2])
    
for i in range(10):
    print('data: {}, label {}'.format(data[i], label[i]))

    
    
with open(train_dataset, 'w') as csvfile:
    writer = csv.writer(csvfile)
    for i in range(samples-test_samples):
        writer.writerow(label[i] + data[i])
    print('train data is written')
        
with open(test_dataset, 'w') as csvfile:
    writer = csv.writer(csvfile)
    for i in range(test_samples):
        writer.writerow(label[i] + data[i])
    print('test data is written')

data: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], label [0]
data: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5], label [1]
data: [9, 8, 7, 6, 5, 4, 3, 2, 1, 0], label [2]
data: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], label [0]
data: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5], label [1]
data: [9, 8, 7, 6, 5, 4, 3, 2, 1, 0], label [2]
data: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], label [0]
data: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5], label [1]
data: [9, 8, 7, 6, 5, 4, 3, 2, 1, 0], label [2]
data: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], label [0]
train data is written
test data is written


In [5]:
import pandas as pd
train_df = pd.read_csv('./ffnn_multiclass_dataset/train_dataset.csv', header = None)
test_df = pd.read_csv('./ffnn_multiclass_dataset/test_dataset.csv', header = None)
train_df.tail()

,0,1,2,3,4,5,6,7,8,9,10
895,1,5,5,5,5,5,5,5,5,5,5
896,2,9,8,7,6,5,4,3,2,1,0
897,0,0,1,2,3,4,5,6,7,8,9
898,1,5,5,5,5,5,5,5,5,5,5
899,2,9,8,7,6,5,4,3,2,1,0


In [6]:
test_df.tail()

,0,1,2,3,4,5,6,7,8,9,10
95,2,9,8,7,6,5,4,3,2,1,0
96,0,0,1,2,3,4,5,6,7,8,9
97,1,5,5,5,5,5,5,5,5,5,5
98,2,9,8,7,6,5,4,3,2,1,0
99,0,0,1,2,3,4,5,6,7,8,9


In [7]:
tf.reset_default_graph()
trainset = tf.contrib.data.TextLineDataset(train_dataset).batch(10)
testset = tf.contrib.data.TextLineDataset(test_dataset).batch(10)

train_itr = trainset.make_one_shot_iterator()
test_itr = testset.make_one_shot_iterator()

train_batch = train_itr.get_next()
test_batch = test_itr.get_next()

decoded_train = tf.decode_csv(train_batch, [[0]]*11)
decoded_test = tf.decode_csv(test_batch, [[0]]*11)

train_label = tf.one_hot(decoded_train[0], depth=3, axis=-1, dtype=tf.float32)
test_label = tf.reshape(decoded_test[0], [-1, 1])

train_data = tf.stack(decoded_train[1:], axis=1)
test_data = tf.stack(decoded_test[1:], axis=1)

test_data = tf.cast(test_data, tf.float32)
train_data = tf.cast(train_data, tf.float32)

print(train_data)
print(test_data)
print(train_label)
print(test_label)

Instructions for updating:
Use `tf.data.TextLineDataset`.
Tensor("Cast_1:0", shape=(?, 10), dtype=float32)
Tensor("Cast:0", shape=(?, 10), dtype=float32)
Tensor("one_hot:0", shape=(?, 3), dtype=float32)
Tensor("Reshape:0", shape=(?, 1), dtype=int32)


In [4]:
def multi_class_model(x, activation, reuse=False):
    layer1 = tf.layers.dense(x, 10, activation=activation, reuse=reuse, name='layer1')
    layer2 = tf.layers.dense(layer1, 10, activation=activation, reuse=reuse, name='layer2')
    layer3 = tf.layers.dense(layer2, 10, activation=activation, reuse=reuse, name='layer3')
    layer4 = tf.layers.dense(layer3, 10, activation=activation, reuse=reuse, name='layer4')
    return tf.layers.dense(layer4, 3, activation=activation, reuse=reuse, name='layer_out')

train_out = multi_class_model(train_data, tf.nn.sigmoid)
test_out = multi_class_model(test_data, tf.nn.sigmoid, True)

for var in tf.trainable_variables():
    print(var)

NameError: name 'train_data' is not defined

In [12]:
loss = tf.losses.softmax_cross_entropy(train_label, train_out)
train_op = tf.train.GradientDescentOptimizer(1e-6).minimize(loss)

In [13]:
pred = tf.nn.softmax(test_out)
accuracy = tf.metrics.accuracy(test_label, tf.argmax(pred, axis=1))

In [14]:
saver = tf.train.Saver()

In [15]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    for i in range(10):
        while True:
            try:
                _, _loss = sess.run([train_op, loss])
                _acc = sess.run(accuracy)
            except tf.errors.OutOfRangeError:
                break
                
        print('epoch: {}, loss: {}, acc: {}'.format(i, _loss, _acc[0]))
        saver.save(sess, './logs/model')

epoch: 0, loss: 1.1010239124298096, acc: 0.3333333432674408
epoch: 1, loss: 1.1008332967758179, acc: 0.3333333432674408
epoch: 2, loss: 1.1377357244491577, acc: 0.3333333432674408
epoch: 3, loss: 1.1010239124298096, acc: 0.3333333432674408
epoch: 4, loss: 1.1008331775665283, acc: 0.3333333432674408
epoch: 5, loss: 1.1377357244491577, acc: 0.3333333432674408
epoch: 6, loss: 1.1010239124298096, acc: 0.3333333432674408
epoch: 7, loss: 1.1008332967758179, acc: 0.3333333432674408
epoch: 8, loss: 1.1377357244491577, acc: 0.3333333432674408
epoch: 9, loss: 1.1010239124298096, acc: 0.3333333432674408
